# Fitting

In [1]:
import sys
sys.path.insert(0, '../../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle as pkl
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from kraus_channels import KrausMap
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss
from optimization import ModelSPAM, ModelQuantumMap, Logger, model_saver
from quantum_channel import DilutedChannel, ChoiMapStatic
from quantum_tools import apply_unitary, measurement, resample, state_fidelity
from experimental import counts_to_probs, generate_pauliInput_circuits, generate_pauli_circuits, marginalize_counts
from spam import SPAM, InitialState, POVMwQR as POVM, IdealPOVM, IdealInit, povm_fidelity, CorruptionMatrix, IdealSPAM
from synthetic_data import generate_spam_data, generate_map_data
from utils import saver, loader, train_val_split
from quantum_circuits import pqc_basic
from spectrum import channel_spectrum, complex_spacing_ratio, Spectrum, coat_spectrum, spectrum_to_angular, spectrum_to_radial
from qiskit.circuit.library import iSwapGate
from analysis import model_to_csr, scatterplot_stats


#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Retrieve CSR

### L = 5

In [2]:
model_list = pkl.load(open(f'../models/integrable_model_4_L=5.model', 'rb'))

csr_integrable_list = model_to_csr(model_list)
radial_list = spectrum_to_radial(csr_integrable_list)
angular_list = spectrum_to_angular(csr_integrable_list)

r_mean_L5, a_mean_L5 = scatterplot_stats(radial_list, angular_list)

In [3]:
model_list = pkl.load(open(f'../models/integrable_model_4_L=10.model', 'rb'))

csr_integrable_list = model_to_csr(model_list)
radial_list = spectrum_to_radial(csr_integrable_list)
angular_list = spectrum_to_angular(csr_integrable_list)

r_mean_L10, a_mean_L10 = scatterplot_stats(radial_list, angular_list)

In [4]:
model_list = pkl.load(open(f'../models/integrable_model_4_L=20_reorder.model', 'rb'))

csr_integrable_list = model_to_csr(model_list)
radial_list = spectrum_to_radial(csr_integrable_list)
angular_list = spectrum_to_angular(csr_integrable_list)

r_mean_L20, a_mean_L20 = scatterplot_stats(radial_list, angular_list)

In [5]:
model_list = pkl.load(open(f'../models/integrable_model_4_L=30_reorder.model', 'rb'))

csr_integrable_list = model_to_csr(model_list)
radial_list = spectrum_to_radial(csr_integrable_list)
angular_list = spectrum_to_angular(csr_integrable_list)

r_mean_L30, a_mean_L30 = scatterplot_stats(radial_list, angular_list)

In [6]:
model_list = pkl.load(open(f'../models/integrable_model_4_L=40_reorder.model', 'rb'))

csr_integrable_list = model_to_csr(model_list)
radial_list = spectrum_to_radial(csr_integrable_list)
angular_list = spectrum_to_angular(csr_integrable_list)

r_mean_L40, a_mean_L40 = scatterplot_stats(radial_list, angular_list)

In [7]:
model_list = pkl.load(open(f'../models/integrable_model_4_L=50_reorder.model', 'rb'))

csr_integrable_list = model_to_csr(model_list)
radial_list = spectrum_to_radial(csr_integrable_list)
angular_list = spectrum_to_angular(csr_integrable_list)

r_mean_L50, a_mean_L50 = scatterplot_stats(radial_list, angular_list)

## Book Keeping

In [10]:
r_AI, a_AI = 0.730666908336575, 0.19849352312582436
r_FF, a_FF = 0.6177348961486886, -0.09610440656318214

r_mean = [r_mean_L5, r_mean_L10, r_mean_L20, r_mean_L30, r_mean_L40, r_mean_L50]
r_theory = [r_AI, r_FF]

a_mean = [a_mean_L5, a_mean_L10, a_mean_L20, a_mean_L30, a_mean_L40, a_mean_L50]
a_theory = [a_AI, a_FF]

In [11]:
marginals_mean_experimental = np.array([r_mean, a_mean]).T
np.savetxt("bookkeeping/figure3/scatterplot_mean.txt", marginals_mean_experimental)

marginals_theory = np.array([r_theory, a_theory]).T
np.savetxt("bookkeeping/figure3/scatterplot_theory.txt", marginals_theory)